# Phase 3: Enhanced Overfitting Prevention

In [1]:
# from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models, regularizers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

## Constants and Data Paths

In [2]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
NUM_CLASSES = 10

# Directories (replace with your dataset paths)
train_dir = './dataset/train'
val_dir = './dataset/val'
test_dir = './dataset/test'

## Enhanced Data Generators

In [3]:
def create_optimized_data_generators():
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        brightness_range=[0.8, 1.2],
        fill_mode='nearest'
    )

    val_test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )
    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )
    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )
    return train_generator, val_generator, test_generator

train_generator, val_generator, test_generator = create_optimized_data_generators()

Found 16938 images belonging to 10 classes.
Found 2425 images belonging to 10 classes.
Found 4846 images belonging to 10 classes.


## Calculate Class Weights

In [4]:
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = {i: class_weights[i] for i in range(NUM_CLASSES)}

## Retrain Phase 1 Model

In [5]:
def retrain_phase_1_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.5),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])

    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

phase_1_model = retrain_phase_1_model()

## Train Phase 1 Model

In [6]:
def train_model(model, train_gen, val_gen, epochs=30, class_weights=None):
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=epochs,
        callbacks=[early_stopping, reduce_lr],
        class_weight=class_weights
    )
    return history

history_phase_1 = train_model(phase_1_model, train_generator, val_generator, class_weights=class_weights)
phase_1_model.save('phase_1_overfit_prevented_model.keras')

Epoch 1/30
530/530 [==============================] - 551s 1s/step - loss: 3.2841 - accuracy: 0.2240 - val_loss: 2.8160 - val_accuracy: 0.3505 - lr: 1.0000e-04
Epoch 2/30
530/530 [==============================] - 402s 757ms/step - loss: 2.9162 - accuracy: 0.2948 - val_loss: 2.8046 - val_accuracy: 0.3584 - lr: 1.0000e-04
Epoch 3/30
530/530 [==============================] - 409s 772ms/step - loss: 2.7541 - accuracy: 0.3271 - val_loss: 2.6041 - val_accuracy: 0.3951 - lr: 1.0000e-04
Epoch 4/30
530/530 [==============================] - 411s 775ms/step - loss: 2.6297 - accuracy: 0.3409 - val_loss: 2.4360 - val_accuracy: 0.4095 - lr: 1.0000e-04
Epoch 5/30
530/530 [==============================] - 408s 769ms/step - loss: 2.4735 - accuracy: 0.3670 - val_loss: 2.4039 - val_accuracy: 0.3975 - lr: 1.0000e-04
Epoch 6/30
530/530 [==============================] - 410s 774ms/step - loss: 2.3539 - accuracy: 0.3871 - val_loss: 2.1033 - val_accuracy: 0.4713 - lr: 1.0000e-04
Epoch 7/30
530/530 [=====

## Retrain Phase 2 Model

In [7]:
def retrain_phase_2_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.5),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])

    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

phase_2_model = retrain_phase_2_model()

## Train Phase 2 Model

In [8]:
history_phase_2 = train_model(phase_2_model, train_generator, val_generator, class_weights=class_weights)
phase_2_model.save('phase_2_overfit_prevented_model.keras')

Epoch 1/30
530/530 [==============================] - 357s 670ms/step - loss: 2.8095 - accuracy: 0.1951 - val_loss: 2.4659 - val_accuracy: 0.2866 - lr: 1.0000e-04
Epoch 2/30
530/530 [==============================] - 344s 649ms/step - loss: 2.5839 - accuracy: 0.2346 - val_loss: 2.3875 - val_accuracy: 0.3039 - lr: 1.0000e-04
Epoch 3/30
530/530 [==============================] - 342s 644ms/step - loss: 2.4635 - accuracy: 0.2744 - val_loss: 2.2359 - val_accuracy: 0.3748 - lr: 1.0000e-04
Epoch 4/30
530/530 [==============================] - 428s 807ms/step - loss: 2.3775 - accuracy: 0.2965 - val_loss: 2.1881 - val_accuracy: 0.3975 - lr: 1.0000e-04
Epoch 5/30
530/530 [==============================] - 371s 700ms/step - loss: 2.2779 - accuracy: 0.3221 - val_loss: 2.2504 - val_accuracy: 0.3616 - lr: 1.0000e-04
Epoch 6/30
530/530 [==============================] - 373s 703ms/step - loss: 2.1931 - accuracy: 0.3383 - val_loss: 2.0007 - val_accuracy: 0.4318 - lr: 1.0000e-04
Epoch 7/30
530/530 [==

## Test and Evaluate Models

In [9]:
def evaluate_model(model, test_gen):
    test_loss, test_accuracy = model.evaluate(test_gen)
    print(f"Test Loss: {test_loss}")
    print(f"Test Accuracy: {test_accuracy}")

evaluate_model(phase_1_model, test_generator)
evaluate_model(phase_2_model, test_generator)

152/152 [==============================] - 23s 153ms/step - loss: 1.4744 - accuracy: 0.5716
Test Loss: 1.4743930101394653
Test Accuracy: 0.5716054439544678
152/152 [==============================] - 21s 139ms/step - loss: 1.5577 - accuracy: 0.5194
Test Loss: 1.557662844657898
Test Accuracy: 0.5193974375724792
